In [27]:
import pysam as ps
import pymongo

In [30]:
client = pymongo.MongoClient("localhost", 27017)
db = client.hospital
mongo_variants = db.variants
mongo_variants.drop()

In [173]:
file = "/Users/mattsdwatson/Documents/KU Leuven/Courses/Semester 2" \
       " 2020/Management of Large-Scale Omics Data/Practical/Practical_3/chr1.vcf.gz"
data = ps.VariantFile(file)

In [32]:
def filter_variants(vcf):
    list = []
    for i, record in enumerate(vcf):
        for sample_id, val in record.samples.items():
            genotype = val["GT"]
            if genotype != (0, 0) and sample_id is not None:
                list.append({sample_id: genotype, "chr": record.chrom, "pos": record.pos, 
                             "ref": record.ref, "alt": record.alts})
    return list

In [33]:
def into_mongodb(collection, file, num_records=1000000):
    data = ps.VariantFile(file)
    filtered = filter_variants(data)[:num_records]
    collection.insert_many(filtered)
    

In [34]:
into_mongodb(mongo_variants, file)

In [35]:
print(mongo_variants.count_documents({}))

1000000


In [36]:
print(mongo_variants.find_one({"pos": {"$gt": 10_000_000}}))

{'_id': ObjectId('5ed6b572bdf4b3fb76a3666e'), 'HG00096': [1, 0], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}


In [37]:
def get_variants(collection, chrom, position, genotype=None):
    """Implement a function that returns all info on variants at `chrom` and `pos`.
    Args:
        collection    MongoDB collection
        chrom         chromosome
        pos           position
        genotype      pair like (1, 0) or (1, 1). If None all are returned
    """
    if genotype is not None:
        genotypes = [genotype, genotype[::-1]]
        for x in collection.find({"$and": [{"pos": {'$eq': position}}, {"chr": {'$eq': chrom}}, {"genotype": {'$in': genotypes}}]}):
            print(x)
    else:
        for x in collection.find({"$and": [{"pos": {'$eq': position}}, {"chr": {'$eq': chrom}}]}):
            print(x)

In [38]:
get_variants(mongo_variants, chrom="1", position=10000400)

{'_id': ObjectId('5ed6b572bdf4b3fb76a3666e'), 'HG00096': [1, 0], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a3666f'), 'HG00099': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36670'), 'HG00100': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36671'), 'HG00101': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36672'), 'HG00102': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36673'), 'HG00103': [0, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36674'), 'HG00105': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36675'), 'HG00106': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a366

In [39]:
# this query does not return any records at the position, as expected
get_variants(mongo_variants, chrom="1", position=10000300)

In [40]:
def get_variants_in_range(collection, chrom, start, end):
    """Returns list of variant informations at `chrom` and from position `start` to `end`.
    Args:
        collection    MongoDB collection
        chrom         chromosome
        start         starting position (inclusive)
        end           end position (inclusive)
    """
    for x in collection.find({"$and": [{"pos": {"$gte": start, "$lte": end}}, {"chr": {'$eq': chrom}}]}):
        print(x)

In [41]:
get_variants_in_range(mongo_variants, chrom='1', start=10_000_000, end=10_003_000)

{'_id': ObjectId('5ed6b572bdf4b3fb76a3666e'), 'HG00096': [1, 0], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a3666f'), 'HG00099': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36670'), 'HG00100': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36671'), 'HG00101': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36672'), 'HG00102': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36673'), 'HG00103': [0, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36674'), 'HG00105': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36675'), 'HG00106': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a366

In [42]:
db.variants.create_index([("chr", pymongo.ASCENDING)])
db.variants.create_index([("pos", pymongo.ASCENDING)])
print(db.variants.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'hospital.variants'}, 'chr_1': {'v': 2, 'key': [('chr', 1)], 'ns': 'hospital.variants'}, 'pos_1': {'v': 2, 'key': [('pos', 1)], 'ns': 'hospital.variants'}}


In [43]:
%time var10m = get_variants_in_range(mongo_variants, chrom='1', start=10_000_000, end=10_003_000)

{'_id': ObjectId('5ed6b572bdf4b3fb76a3666e'), 'HG00096': [1, 0], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a3666f'), 'HG00099': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36670'), 'HG00100': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36671'), 'HG00101': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36672'), 'HG00102': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36673'), 'HG00103': [0, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36674'), 'HG00105': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a36675'), 'HG00106': [1, 1], 'chr': '1', 'pos': 10000400, 'ref': 'T', 'alt': ['A']}
{'_id': ObjectId('5ed6b572bdf4b3fb76a366

In [174]:
# we want to see how many unique reference positions there are in the vcf file
references = []
for record in data: 
   if record.ref not in references: 
    references.append(record.ref)
    
print("Total unique references: {}".format(len(references)))

Total unique references: 17150


In [175]:
# see how many different alternate variants there are for an input of reference positions
# that have been added to the database
def print_variant_numbers(database, reference): 
    for i in list(reference): 
        dict = database.aggregate([{'$match':   
          {'ref': i}}, 
          {'$group': {'_id': "$alt"}},
          {'$count': 'Unique Variants for reference {}'.format(i)}])
        print(list(dict))

In [176]:
# we can see the output for the 4 nucleotides/canonical reference positions, plus a different reference
nucleotides = ["A", "T", "C", "G", "AAT"]
print_variant_numbers(db.variants, reference = nucleotides)

[{'Unique Variants for reference A': 531}]
[{'Unique Variants for reference T': 524}]
[{'Unique Variants for reference C': 529}]
[{'Unique Variants for reference G': 451}]
[{'Unique Variants for reference AAT': 2}]
